환경 변수 설정

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

라이브러리

In [43]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

도구 호출 - tavily_search_func

In [44]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

도구 호출 - wiki_summary

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from langchain_openai import ChatOpenAI

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3,
)

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"커피의 역사"})
pprint(summarized_text)

('The history of coffee began in East Africa, specifically in Ethiopia, where '
 'coffee plants were first discovered. The story goes that a goatherd named '
 'Kaldi noticed that his goats became more energetic after eating the red '
 'berries of a certain plant. Kaldi tried the berries himself and felt '
 'invigorated, leading him to introduce them to the local monastery. The monks '
 'began to cultivate the plant and use its beans to stay awake during long '
 'hours of prayer.\n'
 '\n'
 'From Ethiopia, coffee spread to the Arabian Peninsula, where it was '
 'cultivated and traded throughout the Middle East. The Arabs were responsible '
 'for the first commercial cultivation of coffee and for developing many of '
 'the techniques that are still used today, such as roasting and grinding the '
 'beans.\n'
 '\n'
 'Coffee was introduced to Europe in the 17th century, where it became popular '
 'in countries such as Italy, France, and England. The first coffeehouses were '
 'established in

In [46]:
from pydantic import BaseModel, Field


# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

도구 호출 - db_search_cafe_func

In [47]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [48]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...

메뉴 번호: 2
메뉴 이름: 카페라떼
내용:
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다...


In [49]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
cafe_db.save_local("../db/cafe_db")


# Retriever 생성
menu_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 6},
)

# 쿼리 테스트
query = "우유 포함 메뉴의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 6개
메뉴 번호: 2
메뉴 이름: 카페라떼

메뉴 번호: 3
메뉴 이름: 카푸치노

메뉴 번호: 7
메뉴 이름: 프라푸치노

메뉴 번호: 4
메뉴 이름: 바닐라 라떼

메뉴 번호: 8
메뉴 이름: 녹차 라떼

메뉴 번호: 10
메뉴 이름: 티라미수



In [50]:
from langchain_core.tools import tool
from typing import List

# cafe_db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_cafe_func))
print("-"*100)

print("name: ")
print(db_search_cafe_func.name)
print("-"*100)

print("description: ")
pprint(db_search_cafe_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_cafe_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_cafe_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_cafe_func',
 'type': 'object'}


few-shot 도구 호출

In [52]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime


examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요", name="example_user"),

    AIMessage("아메리카노의 메뉴 정보를 검색하고, 커피의 역사에 대해 위키피디아에서 찾아보겠습니다.", name="example_assistant"),

    # 아메리카노 메뉴 검색
    AIMessage("", name="example_assistant", tool_calls=[
        {"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}
    ]),
    ToolMessage("아메리카노: 가격 ₩4,500, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.", tool_call_id="1"),

    # 커피 역사 검색
    AIMessage("이제 커피의 역사에 대해 위키피디아에서 알아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[
        {"name": "wiki_summary", "args": {"query": "커피", "k": 1}, "id": "2"}
    ]),
    ToolMessage("커피는 에티오피아 고원지대에서 기원한 것으로 알려져 있으며, 15세기 아라비아 반도에서 본격적인 음료 문화가 시작되었습니다. 이후 유럽과 전 세계로 확산되며 다양한 커피 문화가 발전했습니다.", tool_call_id="2"),

    # 종합 응답
    AIMessage(
        "아메리카노는 ₩4,500이며, 에스프레소 샷 2개에 뜨거운 물을 추가해 만든 커피로 깔끔하고 산뜻한 맛이 특징입니다. \
커피는 에티오피아에서 기원하여 15세기 아라비아 반도에서 본격적으로 음료로 마시기 시작했고, 이후 유럽과 전 세계로 확산되며 다양한 커피 문화가 형성되었습니다.",
        name="example_assistant"
    )
]

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")



system = """You are an AI assistant that provides information about café menu items and general knowledge related to food and beverages.

Tool Usage Guidelines:

db_search_cafe_func: For café menu information (price, ingredients, description)

wiki_summary: For general knowledge (history, preparation methods, cultural background)

tavily_search_func: For up-to-date information (trends, news, real-time updates)

Usage Principles:

For café menu-related questions → Always search the menu database first.

For historical, cultural, or general knowledge → Use Wikipedia summaries.

For current trends or news → Use web search.

For multi-faceted questions → Use tools sequentially as needed.

Clearly distinguish the sources of information in your response.

Purpose: These principles help you choose the appropriate tool based on the context of the user's question.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3,
)

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary, db_search_cafe_func])

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [53]:
from langchain_core.runnables import RunnableConfig, chain

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)
    


db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '라둥 디저트'}, 'id': 'tkv0e79ep', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='fa5b49ee-9a7b-4a84-845f-b73d8b7d9ff9', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 10, 'menu_name': '티라미수'}, page_content='10. 티라미수\\n    • 가격: ₩7,500\\n    • 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더\\n    • 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.'), Document(id='dfa4a838-cead-48e9-8e8f-9fde26208f90', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 4, 'menu_name': '바닐라 라떼'}, page_content='4. 바닐라 라떼\\n   • 가격: ₩6,000\\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.')]", name='db_search_cafe_func', tool_call_id='tkv0e79ep')]
-----

In [54]:
# 응답 출력 
pprint(response.content)

('카페라떼와 어울리는 디저트는 티라미수입니다. 가격은 7,500원이며, 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더로 만들어진 '
 '이탈리아 전통 디저트입니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.\n'
 '\n'
 '이제 라떼의 유래에 대해 알려드리겠습니다.')
